In [ ]:
from gtts import gTTS
import os

text = "شو هي الأسماء اللي علمها الله سبحانه وتعالى للنبي آدم؟"

# Create TTS object
tts = gTTS(text=text, lang="ar")

tts.save("output.mp3")

os.system("start output.mp3")

0

In [ ]:
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
from elevenlabs import play
import os

load_dotenv()

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

audio = elevenlabs.text_to_speech.convert(
    text="شو هي الأسماء اللي علمها الله سبحانه وتعالى للنبي آدم؟",
    voice_id="JBFqnCBsd6RMkjVDRZzb",
    model_id="eleven_multilingual_v2",
    output_format="mp3_44100_128",
)

from pathlib import Path

# Convert generator to bytes
audio_bytes = b"".join(audio)

with open("output2.mp3", "wb") as f:
    f.write(audio_bytes)

print("Audio saved as output2.mp3")


Audio saved as output2.mp3


ModuleNotFoundError: No module named 'playsound'

In [ ]:
elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

elevenlabs.voices.delete(voice_id="9hpTsoJZGZdxxmZyoOPp")

ApiError: headers: {'date': 'Sun, 31 Aug 2025 12:49:25 GMT', 'server': 'uvicorn', 'content-length': '142', 'content-type': 'application/json', 'access-control-allow-origin': '*', 'access-control-allow-headers': '*', 'access-control-allow-methods': 'POST, PATCH, OPTIONS, DELETE, GET, PUT', 'access-control-max-age': '600', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-trace-id': '1946a1466de8b38541b51393f801fff0', 'x-region': 'us-central1', 'via': '1.1 google, 1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 401, body: {'detail': {'status': 'missing_permissions', 'message': 'The API key you used is missing the permission voices_write to execute this operation.'}}

In [18]:
# example.py
import os
import time
import base64
from contextlib import ExitStack
from io import BytesIO
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs

load_dotenv()

elevenlabs = ElevenLabs(
  api_key=os.getenv("ELEVENLABS_API_KEY"),
)

voice = elevenlabs.voices.pvc.create(
    name="My Professional Voice Clone",
    language="en",
    description="A professional voice clone of my voice"
)

print(voice)


ApiError: headers: {'date': 'Sun, 31 Aug 2025 12:34:43 GMT', 'server': 'uvicorn', 'content-length': '142', 'content-type': 'application/json', 'access-control-allow-origin': '*', 'access-control-allow-headers': '*', 'access-control-allow-methods': 'POST, PATCH, OPTIONS, DELETE, GET, PUT', 'access-control-max-age': '600', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-trace-id': '0924f36f1d56a137e8182296708803c5', 'x-region': 'us-central1', 'via': '1.1 google, 1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 400, body: {'detail': {'status': 'professional_voice_limit_reached', 'message': 'You have reached your maximum amount of professional voice slots (1 / 1).'}}

In [20]:
print(voice.voice_id)

9hpTsoJZGZdxxmZyoOPp


In [ ]:
# Define the list of file paths explicitly
# Replace with the paths to your audio and/or video files.
# The more files you add, the better the clone will be.
sample_file_paths = [
    r"C:\Users\Lenovo\Downloads\project\sayed_clips\sayed_01.wav",
    r"C:\Users\Lenovo\Downloads\project\sayed_clips\sayed_02.wav",
    r"C:\Users\Lenovo\Downloads\project\sayed_clips\sayed_03.wav"
]

samples = None

files_to_upload = []
# Use ExitStack to manage multiple open files
with ExitStack() as stack:
    for filepath in sample_file_paths:
        # Open each file and add it to the stack
        audio_file = stack.enter_context(open(filepath, "rb"))
        filename = os.path.basename(filepath)

        # Create a File object for the SDK
        files_to_upload.append(
            BytesIO(audio_file.read())
        )

    samples = elevenlabs.voices.pvc.samples.create(
        voice_id=voice.voice_id,
        files=files_to_upload # Pass the list of File objects
    )


ApiError: headers: {'date': 'Sun, 31 Aug 2025 12:17:16 GMT', 'server': 'uvicorn', 'content-length': '214', 'content-type': 'application/json', 'access-control-allow-origin': '*', 'access-control-allow-headers': '*', 'access-control-allow-methods': 'POST, PATCH, OPTIONS, DELETE, GET, PUT', 'access-control-max-age': '600', 'strict-transport-security': 'max-age=31536000; includeSubDomains', 'x-trace-id': 'a3c5abd27e9349a9e9442eaa9e5dfdab', 'x-region': 'us-central1', 'via': '1.1 google, 1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 400, body: {'detail': {'status': 'language_mismatch', 'message': 'The language of the audio samples does not match provided language. This can happen if the sample is low quality. Please try again with a higher quality sample.'}}

In [ ]:
sample_ids_to_check = []
for sample in samples:
    if sample.sample_id:
        print(f"Starting separation for sample: {sample.sample_id}")
        elevenlabs.voices.pvc.samples.speakers.separate(
            voice_id=voice.voice_id,
            sample_id=sample.sample_id
        )
        sample_ids_to_check.append(sample.sample_id)

while sample_ids_to_check:
    # Create a copy of the list to iterate over, so we can remove items from the original
    ids_in_batch = list(sample_ids_to_check)
    for sample_id in ids_in_batch:
        status_response = elevenlabs.voices.pvc.samples.speakers.get(
            voice_id=voice.voice_id,
            sample_id=sample_id
        )
        status = status_response.status
        print(f"Sample {sample_id} status: {status}")
        if status == "completed" or status == "failed":
            sample_ids_to_check.remove(sample_id)

    if sample_ids_to_check:
        # Wait before the next poll cycle
        time.sleep(5) # Wait for 5 seconds

print("All samples have been processed or removed from polling.")
